<a href="https://colab.research.google.com/github/ankitk75/AIML-Project-Series/blob/main/Admission_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import csv
import pandas as pd

# Define the input and output file paths
input_file_path = '/content/input.txt'
output_file_path = '/content/output.csv'

# Initialize lists to hold questions and answers
questions = []
answers = []

# Read the input file and parse the content
with open(input_file_path, 'r') as file:
    lines = file.readlines()
    for line in lines:
        if line.strip():  # Skip any empty lines
            question, answer = line.split(',', 1)  # Split only at the first comma
            questions.append(question.strip())
            answers.append(answer.strip())

# Write the questions and answers to a new CSV file
with open(output_file_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['question', 'answer'])  # Write the header row
    for question, answer in zip(questions, answers):
        csvwriter.writerow([question, answer])


In [12]:
def load_dataset_from_csv(csv_path):
    df = pd.read_csv(csv_path)
    dialogues = [{"input": row['question'], "output": row['answer']} for _, row in df.iterrows()]
    return Dataset.from_pandas(pd.DataFrame(dialogues))


# dataset = load_dataset_from_csv('output.csv')

In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00


In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

# Add padding token
tokenizer.pad_token = tokenizer.eos_token

# Prepare the dataset
dataset = load_dataset_from_csv('output.csv')
train_test_split = dataset.train_test_split(test_size=0.1)
dataset = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})


# Tokenize the dataset
def tokenize_function(examples):
    inputs = tokenizer(examples['input'], truncation=True, padding='max_length', max_length=128)
    outputs = tokenizer(examples['output'], truncation=True, padding='max_length', max_length=128)
    return {'input_ids': inputs['input_ids'], 'attention_mask': inputs['attention_mask'],
            'labels': outputs['input_ids']}


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/49 [00:00<?, ? examples/s]

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

In [5]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [14]:
# Tokenize the dataset
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',
    num_train_epochs=15,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,8.563600,8.281356
2,7.168700,3.015113
3,4.347100,2.655470
4,2.798700,2.313536
5,2.650200,2.174286
6,2.358300,2.030334
7,2.205400,1.929954
8,2.020600,1.821768
9,1.962100,1.757895
10,1.714700,1.689460


TrainOutput(global_step=195, training_loss=2.7521566928961336, metrics={'train_runtime': 104.6565, 'train_samples_per_second': 7.023, 'train_steps_per_second': 1.863, 'total_flos': 170648752619520.0, 'train_loss': 2.7521566928961336, 'epoch': 15.0})

In [15]:
model.save_pretrained("fine-tuned-dialoGPT-medium")
tokenizer.save_pretrained("fine-tuned-dialoGPT-medium")

('fine-tuned-dialoGPT-medium/tokenizer_config.json',
 'fine-tuned-dialoGPT-medium/special_tokens_map.json',
 'fine-tuned-dialoGPT-medium/vocab.json',
 'fine-tuned-dialoGPT-medium/merges.txt',
 'fine-tuned-dialoGPT-medium/added_tokens.json',
 'fine-tuned-dialoGPT-medium/tokenizer.json')

In [21]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("fine-tuned-dialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("fine-tuned-dialoGPT-medium")

def run(user_text, chat_history_ids):
    input_ids = tokenizer.encode(user_text + tokenizer.eos_token, return_tensors='pt')
    if chat_history_ids is None:
        bot_history_ids = input_ids
    else:
        bot_history_ids = torch.cat([chat_history_ids, input_ids], dim=-1)

    chat_history_ids = model.generate(bot_history_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    resp = tokenizer.decode(chat_history_ids[:, bot_history_ids.shape[-1]:][0], skip_special_tokens=True)

    return resp, chat_history_ids


In [26]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


# tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
# model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small")

tokenizer = AutoTokenizer.from_pretrained("fine-tuned-dialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("fine-tuned-dialoGPT-medium")


# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens,
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))


>> User:how to get admission in MIT?
DialoGPT: You need to be accepted by MIT to apply
>> User:What are the hostel facilities at Manipal Institute of Technology?
DialoGPT: The hostel is in the main building of the campus
>> User:Can international students apply for B.Tech at Manipal Institute of Technology?
DialoGPT: Yes, international students can apply for B.Tech at Manipal Institute of Technology
>> User:What is the selection process for BTech at MIT Manipal?
DialoGPT: There is a selection process for all courses at MIT
>> User:thank you
DialoGPT: You are welcome
